In [1]:
import numpy as np

DIMENSIONS = (36, 36)

## Turn asteroid belt into 2D numpy array in which 1 represents an asteroid

In [2]:
with open("./input.txt") as f:
    asteroid_belt = f.read().replace("\n", "")

asteroid_belt = asteroid_belt.replace(".", "0").replace("#", "1")
asteroid_belt = np.array(list(map(int, list(asteroid_belt))))
asteroid_belt.shape = (36, 36)
asteroid_belt = asteroid_belt.transpose()

## Store (x, y) of asteroid locations in an list

In [3]:
asteroid_locs = [
    (x, -1*y) for x in range(36)
    for y in range(36)
    if asteroid_belt[x, y] == 1
]

In [4]:
from functools import partial
from typing import Tuple

round_to_five_decimal = partial(round, ndigits=5)

def get_unit_vector_between_points(
    point1: Tuple[float, float], point2: Tuple[float, float]
) -> Tuple[float, float]:
    """Returns a vector of unit length between two points, rounded to 5 decimal places"""
    point_diff = np.array(point2) - np.array(point1)
    unit_vec = point_diff / np.linalg.norm(point_diff)

    return tuple(map(round_to_five_decimal, unit_vec))

In [5]:
reachable_asteroids = np.zeros((36, 36))

for asteroid in asteroid_locs:
    # get list of all asteroids other than current one
    other_asteroids = [other_asteroid for other_asteroid in asteroid_locs if asteroid != other_asteroid]
    unit_vector_to_asteroid = set()

    unit_vector_to_asteroid = set([
        get_unit_vector_between_points(asteroid, other_asteroid)
        for other_asteroid in other_asteroids
    ])

    reachable_asteroids[asteroid] = len(unit_vector_to_asteroid)

np.where(reachable_asteroids == np.amax(reachable_asteroids))

(array([17]), array([14]))

In [6]:
asteroid_loc = (17, -22)

other_asteroids = [other_asteroid for other_asteroid in asteroid_locs if other_asteroid != asteroid_loc]

# create a dictionary where key is unit vector to other asteroids
# and value is list of asteroids, sorted by Euclidean distance
unit_vec_to_asteroids = {}

for other_asteroid in other_asteroids:
    unit_vector = get_unit_vector_between_points(asteroid_loc, other_asteroid)
    if unit_vector not in unit_vec_to_asteroids:
        unit_vec_to_asteroids[unit_vector] = []

    unit_vec_to_asteroids[unit_vector].append(other_asteroid)

def get_euclidean_distance(
    point1: Tuple[float, float], point2: Tuple[float, float] = (17, -22)
) -> float:
    """Default is to get Euclidean distance from (17, 22) for distance from our asteroid"""
    return np.linalg.norm(np.array(point2) - np.array(point1))

for unit_vec in unit_vec_to_asteroids.keys():
    unit_vec_to_asteroids[unit_vec].sort(key=get_euclidean_distance)

In [7]:
import math

def complex_num_to_clockwise_angle(complex_num: Tuple[float, float]) -> float:
    """Takes a complex number and returns its 0 to 360 value clockwise

    0 degrees postive Y-axis
    """
    angle = math.degrees(math.atan2(*complex_num))
    if angle < 0:
        angle = 360 + angle
    return angle

In [8]:
asteroids_destroy_order = []
vaporized_asteroids = 0

for other_asteroid in sorted(unit_vec_to_asteroids, key=complex_num_to_clockwise_angle):
    try:
        asteroid_to_vaporize = unit_vec_to_asteroids[other_asteroid].pop(0)
    except IndexError:
        pass
    else:
        vaporized_asteroids += 1
        asteroids_destroy_order.append(asteroid_to_vaporize)


asteroids_destroy_order[199][0]*100 - asteroids_destroy_order[199][1]

1321